In [ ]:
width, height, transform = metadata_from_tile(tile)
bands = ['band_0','band_1','band_2','band_3','band_4','band_5']
#agrandar si tienen mas memoria, corre un poco mas rapido pero ojo que se pueden distorsionar los bordes
windows = sliding_windows(100, 100, width, height)
os.makedirs('./predictions/', exist_ok=True)
out_raster = f'./predictions/{tile}'
with rasterio.open(out_raster, 'w', driver='GTiff', count=1, width=width, height=height, dtype=np.float64, transform=transform, crs=CRS.from_epsg(4326), compress='lzw'
) as dst:
    for window, _ in tqdm(windows):           
        img_df = create_windowed_dataset(tile, window, bands)
        n_bands = img_df.shape[1]
        r,m,n = (n_bands, window.width, window.height)
        #clf es el clasificador que entrenamos antes
        res = clf.predict(img_df).astype(np.float64)
        res = np.expand_dims(res.reshape(m,n), axis=0)
        dst.write(res, window=window)